In [1]:
import sys 
sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from NN import network, activation_functions, loss_functions
from NN import ANN
X = np.array([[1,2],[3,4]])


## First Code running

In [2]:
# Random Init
teste = ANN.ANN("sigmoid")
teste.initialize_random_weights(2, [3], 2)
teste.show_weights()
teste.show_setup()
Y, aY = teste.foward_propagation(X)
print("Y\n",Y)
print("\na(Y)\n",aY)

Hidden Layer  1
[[0.94873394 0.11414162 0.55954911]
 [0.58339539 0.53380147 0.94712821]
 [0.64962293 0.17700522 0.44960808]] 

Output Layer 
[[0.89201452 0.8027576 ]
 [0.98750924 0.8081617 ]
 [0.52182774 0.93853488]
 [0.96373231 0.53303754]] 

 Input size:  2
 Number of hidden layers:  1
 Number of perceptrons at each layer: 
 HL 1: 3
 Number of classes: 2 

Y
 [[3.09026194 2.74137004]
 [3.31240253 2.99915082]]

a(Y)
 [[0.95648927 0.93942411]
 [0.96485185 0.95253575]]


## Comparing with the new Implementation by using the same weights

In [3]:
from importlib import reload
reload(network)
I = np.array([[0.05, 0.10]])
W = np.array([[0.15, 0.2], [0.25, 0.3]]).T
h1 = network.Layer(2, 2, 'sigmoid', weights=W, bias=0.35, label="H1")
netH, outH =  h1.feed_forward(I)
print(netH)
print(outH)

[[0.3775 0.3925]]
[[0.59326999 0.59688438]]


In [4]:
from importlib import reload
reload(network)
I = np.array([[0.59326999, 0.59688438]])
W = np.array([[0.4, 0.45], [0.50, 0.55]]).T
o1 = network.Layer(2, 2, 'sigmoid', weights=W, bias=0.6, label="H1")
netO, outO =  o1.feed_forward(I)
print(netO)
print(outO)

[[1.10590597 1.2249214 ]]
[[0.75136507 0.77292847]]


In [5]:
from importlib import reload
reload(network)
model = network.NN(loss='smd')
X = np.array([[0.05, 0.10]])

Wh = np.array([[0.15, 0.2], [0.25, 0.3]]).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]]).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="H1")
model.add_layer(h)
model.add_layer(o)
model.show_weights()

Y, aY = model.feed_forward(X)
Y_, aY_ = ([1.10590597, 1.2249214 ], [0.75136507, 0.77292847])
print("Y\n",Y_)
print("\na(Y)\n",aY_)

assert((Y - Y_).sum() < np.finfo(np.float32).eps)
assert((aY - aY_).sum() < np.finfo(np.float32).eps)


Y
 [1.10590597, 1.2249214]

a(Y)
 [0.75136507, 0.77292847]


## Working on the back propagation

In [6]:
from importlib import reload
reload(network)
model.show_weights()
Y = np.array([0.01, 0.99])
Etotal = loss_functions.smd(aY, Y)
print(Etotal)
assert((Etotal - 0.2983711) < np.finfo(np.float32).eps)

0.2983711087600027


### Emulating the weights update for the layer O 

In [7]:
reload(loss_functions)
reload(activation_functions)
lr = 0.5
#Done - Partial
dEo_dw = loss_functions.smd_derivative_chain(outO, Y)
print(dEo_dw)

# Done
dOuto_Dneto = activation_functions.sigmoid_derivative_chain(outO)
print('dout',dOuto_Dneto)

# Done - Self.input
dNeto  = outH
print('dnet',dNeto) 
# Done
deltaO  = dEo_dw * dOuto_Dneto
print('delta', deltaO)

dWO = deltaO * outH
print('dw', dWO)
# Done
updateO = Wo - lr * dWO
print('update', updateO)

[[ 0.74136507 -0.21707153]]
dout [[0.1868156  0.17551005]]
dnet [[0.59326999 0.59688438]]
delta [[ 0.13849856 -0.03809824]]
dw [[ 0.08216704 -0.02274024]]
update [[0.35891648 0.51137012]
 [0.40891648 0.56137012]]


In [8]:
#
# Done - Partial
dEo_dOh = deltaO * Wo
dETotal_dOh = dEo_dOh.sum(axis=1)
print (dEo_dOh)
print('dETotal_dOh', dETotal_dOh)

# Done
dOuth_Dneth = activation_functions.sigmoid_derivative_chain(outH)
print('dOuth_Dneth', dOuth_Dneth)


# Done
deltaH = dETotal_dOh * dOuth_Dneth 
print('deltaH', deltaH)

# done
# self.input
dNeth_dw = X
dWh = deltaH * dNeth_dw
print('dWh', dWh)

# Done
updateH = Wh - lr * dWh
print('update', updateH)

[[ 0.05539942 -0.01904912]
 [ 0.06232435 -0.02095403]]
dETotal_dOh [0.03635031 0.04137032]
dOuth_Dneth [[0.24130071 0.24061342]]
deltaH [[0.00877135 0.00995425]]
dWh [[0.00043857 0.00099543]]
update [[0.14978072 0.24950229]
 [0.19978072 0.29950229]]


In [25]:
reload(loss_functions)
reload(activation_functions)
reload(network)
network.DEBUG = False

X = np.array([[0.05, 0.10]], np.float64)

Wh = np.array([[0.15, 0.2], [0.25, 0.3]], np.float64).T
h = network.Layer(2, 2, 'sigmoid', weights=Wh, bias=0.35, label="H1")

Wo = np.array([[0.4, 0.45], [0.50, 0.55]], np.float64).T
o = network.Layer(2, 2, 'sigmoid', weights=Wo, bias=0.6, label="H1")
Y = np.array([0.01, 0.99], np.float64)

for i in range(50000):
    netH, outH = h.feed_forward(X)
#     print(netH, outH)
    netO, outO = o.feed_forward(outH)
#     print(netO, outO)
    
    Etotal = loss_functions.smd(outO, Y)
    if (i % 1000) == 0:
        print(i, 'error', Etotal)
    
    dEo_dw = loss_functions.smd_derivative_chain(outO, Y)
    network.dprint(dEo_dw)
    network.dprint ("")
    network.dprint ("==========================================")
    network.dprint ("Back Propagate Layer O")
    network.dprint ("==========================================")
    o.backpropagate(dETotal_dOut=dEo_dw)
    network.dprint ("==========================================")
    network.dprint ("")

    network.dprint ("==========================================")
    network.dprint ("Back Propagate Layer H")
    network.dprint ("==========================================")
    h.backpropagate(output_layer=o)
    network.dprint ("==========================================")

print (outO)

0 error 0.2983711087600027
1000 error 0.0011202377365627065
2000 error 0.00044785141631775586
3000 error 0.00025349028036380844
4000 error 0.00016571865888432704
5000 error 0.00011734553835230822
6000 error 8.7442721889668e-05
7000 error 6.75166200528868e-05
8000 error 5.351545683205839e-05
9000 error 4.32812038815559e-05
10000 error 3.556829373023961e-05
11000 error 2.961232064457121e-05
12000 error 2.4920877822775443e-05
13000 error 2.1164068391319185e-05
14000 error 1.8113574397803993e-05
15000 error 1.5607002184429376e-05
16000 error 1.3526123820153284e-05
17000 error 1.1783116670472732e-05
18000 error 1.0311590002491457e-05
19000 error 9.060575063147334e-06
20000 error 7.990404544484642e-06
21000 error 7.06982827068485e-06
22000 error 6.273956558913241e-06
23000 error 5.5827692280249095e-06
24000 error 4.980018389688318e-06
25000 error 4.4524099926329095e-06
26000 error 3.988985709203115e-06
27000 error 3.5806508172542315e-06
28000 error 3.2198098315234526e-06
29000 error 2.900082

## Random initialization

### Softmax Sample and Test

In [ ]:
from importlib import reload
reload(network)
model = network.NN(loss='cross_entropy')
model.add_layer(network.Layer(2, 5, 'sigmoid',  label="H1"))
model.add_layer(network.Layer(5, 2, 'sigmoid',  label="output"))
model.show_weights()
X = np.array([[0.05, 0.10]])
Y, aY = model.feed_forward(X)

print("Y\n",Y)
print("\na(Y)\n",aY)

## Testing the loss functions 

### Mean Squared Error

In [ ]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])
lr = 0.01
eps  = 0.01
max_it = 100
for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.mse_derivative(I, aY, Y_)
    W = W - lr * dC

assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)
    


### Cross Entropy 

In [ ]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])

for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.cross_entropy_derivative(I, aY, Y_)
    W = W - lr * dC

assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)

### Sum of Squared Errors 

In [ ]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])

for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.smd_derivative(I, aY, Y_)
    W = W - lr * dC
assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)    